In [13]:
import pandas as pd
import torch
import torch.nn as nn
from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics.pairwise import cosine_similarity
from pandarallel import pandarallel
import joblib

pandarallel.initialize(progress_bar=True)


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [2]:
df = pd.read_pickle("../../data/esm2_embeddings.pkl")
print(df.LABEL.value_counts())
df.head()

LABEL
NEUTRAL    83924
LOF        25376
GOF         3137
Name: count, dtype: int64


,VARIANTKEY,LABEL,ENSG,GENE_SYMBOL,AA_POSITION,PROTEIN_REF,PROTEIN_ALT,REF_EMBEDDING_ESM2,ALT_EMBEDDING_ESM2
0,1-100196274-A-C,LOF,ENSG00000137992,DBT,477,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.35547394, -1.0680466, -5.513677, -0.804930..."
1,1-100196286-T-C,NEUTRAL,ENSG00000137992,DBT,473,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.34180462, -1.1007842, -5.556662, -0.767595..."
2,1-100196349-T-C,LOF,ENSG00000137992,DBT,452,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.37278727, -1.1201291, -5.4481387, -0.74900..."
3,1-100206470-G-A,LOF,ENSG00000137992,DBT,395,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.3627783, -1.0958921, -5.580294, -0.7817215..."
4,1-100206621-C-T,LOF,ENSG00000137992,DBT,345,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.3289509, -1.0981828, -5.574082, -0.8473001..."


In [3]:
# Preprocess the data
label_mapping = {'NEUTRAL': 0, 'LOF': 1, 'GOF': 2}
df['LABEL_MAP'] = df['LABEL'].map(label_mapping)


In [4]:
ref_embedding_df = df['REF_EMBEDDING_ESM2'].apply(pd.Series)
ref_embedding_df.columns = ['ref_' + str(col) for col in ref_embedding_df.columns]

alt_embedding_df = df['ALT_EMBEDDING_ESM2'].apply(pd.Series)
alt_embedding_df.columns = ['alt_' + str(col) for col in alt_embedding_df.columns]

X = pd.concat([ref_embedding_df, alt_embedding_df], axis=1)
y = df['LABEL_MAP']


In [5]:
TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1
VALIDATION_RATIO = VALIDATION_SIZE / (1 - TEST_SIZE)

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=TEST_SIZE, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=VALIDATION_RATIO, random_state=42)

print(f"Training set: {len(X_train)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Testing set: {len(X_test)} samples")
X_train

Training set: 89949 samples
Validation set: 11244 samples
Testing set: 11244 samples


,ref_0,ref_1,ref_2,ref_3,ref_4,ref_5,ref_6,ref_7,ref_8,ref_9,...,alt_1270,alt_1271,alt_1272,alt_1273,alt_1274,alt_1275,alt_1276,alt_1277,alt_1278,alt_1279
20269,-1.677370,-5.069440,1.790775,2.504909,3.339102,-4.563093,2.255173,7.203899,4.016263,2.939487,...,10.755238,-0.974600,-3.884165,-1.225205,-2.143846,-6.034309,4.386026,-1.678212,3.065606,0.464755
25081,-1.102459,-6.107592,3.221740,1.416400,-2.068759,-4.090089,3.213005,8.528033,1.013075,-1.497316,...,6.419495,2.151514,-2.389071,-6.705357,2.629794,-9.097048,-4.480047,4.491002,2.963162,5.034623
18979,-0.350532,3.262130,-0.268965,-0.764486,3.841102,0.458391,3.354928,-0.144458,-0.380587,1.135145,...,5.466431,0.227993,-1.226452,0.696634,-1.394614,1.112188,0.804384,0.141640,-1.183693,-2.372631
82624,-0.395097,-2.786441,1.088749,-1.369367,-1.041126,2.109502,-0.773584,-5.250056,0.325605,2.864200,...,5.880882,0.483550,-2.217463,-2.529668,2.024062,-2.086164,1.594726,6.007213,5.820427,4.075550
12669,0.325165,-0.978003,0.497170,2.514515,1.613455,-4.068345,-0.003553,8.409565,2.345423,2.246047,...,10.033262,2.935265,-7.642428,-2.454281,-2.555973,-3.343869,5.175861,-0.920529,4.225687,-0.346533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105470,2.164968,-4.266226,0.715478,-0.444632,-0.405317,-2.309124,1.186860,-2.859610,2.714831,4.479666,...,3.218880,-0.116098,-3.977753,0.891991,0.875647,-3.317369,0.002371,-2.504501,-1.602196,-0.823146
86161,1.380002,-5.259967,1.026949,3.624580,1.619648,-2.841283,1.728307,16.387716,4.096630,4.054551,...,10.580894,2.409778,-7.797612,-3.986648,-3.250914,-2.082717,2.017985,-1.519410,3.252011,1.121499
107790,0.115311,2.376308,-0.093431,-1.726331,3.014427,-0.113411,3.079608,0.044947,-0.056119,2.475350,...,5.078647,-1.067601,-3.117823,0.428103,-0.222030,-2.276420,1.770209,-0.676130,-0.221560,1.680834
23934,1.872022,0.940849,0.189312,0.657872,-0.037050,-1.835866,0.723961,-1.615863,-0.450074,3.945919,...,3.365461,3.920008,-3.459285,-1.669742,2.974412,-5.431729,1.983080,-1.477734,-2.642721,2.710844


In [6]:
# Train a logistic regression model
linear_model = LogisticRegression()
linear_model.fit(X_train, y_train)
# Evaluate the model
lr_pred = linear_model.predict(X_test)
print(classification_report(y_test, lr_pred, target_names=label_mapping.keys()))

              precision    recall  f1-score   support

     NEUTRAL       0.85      0.92      0.88      8412
         LOF       0.67      0.53      0.59      2543
         GOF       0.50      0.22      0.31       289

    accuracy                           0.81     11244
   macro avg       0.67      0.56      0.59     11244
weighted avg       0.80      0.81      0.80     11244



/Users/victormaricato/mambaforge/envs/lof-gof-prediction/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
# Train a random forest classification model
rf_model = RandomForestClassifier()
rf_model.fit(X_train, y_train)
# Evaluate the model
rf_pred = rf_model.predict(X_test)
print(classification_report(y_test, rf_pred, target_names=label_mapping.keys()))

              precision    recall  f1-score   support

     NEUTRAL       0.91      0.93      0.92      8412
         LOF       0.78      0.71      0.74      2543
         GOF       0.77      0.67      0.72       289

    accuracy                           0.88     11244
   macro avg       0.82      0.77      0.79     11244
weighted avg       0.87      0.88      0.87     11244



In [8]:
# Train an XGBoost model
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
xgb_model.fit(X_train, y_train)

# Evaluate the XGBoost model
xgb_pred = xgb_model.predict(X_test)
print(classification_report(y_test, xgb_pred, target_names=label_mapping.keys()))

              precision    recall  f1-score   support

     NEUTRAL       0.89      0.93      0.91      8412
         LOF       0.76      0.67      0.71      2543
         GOF       0.77      0.58      0.67       289

    accuracy                           0.86     11244
   macro avg       0.81      0.73      0.76     11244
weighted avg       0.86      0.86      0.86     11244



In [9]:
# Train a LightGBM model
lgb_model = LGBMClassifier(n_estimators=100, learning_rate=0.1, random_state=42)
lgb_model.fit(X_train, y_train)

# Evaluate the LightGBM model
lgb_pred = lgb_model.predict(X_test)
print(classification_report(y_test, lgb_pred, target_names=label_mapping.keys()))

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.772452 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 652640
[LightGBM] [Info] Number of data points in the train set: 89949, number of used features: 2560
[LightGBM] [Info] Start training from score -0.291971
[LightGBM] [Info] Start training from score -1.491335
[LightGBM] [Info] Start training from score -3.570628
              precision    recall  f1-score   support

     NEUTRAL       0.89      0.93      0.91      8412
         LOF       0.76      0.67      0.71      2543
         GOF       0.76      0.63      0.69       289

    accuracy                           0.87     11244
   macro avg       0.80      0.74      0.77     11244
weighted avg       0.86      0.87      0.86     11244



In [10]:
# Convert data to PyTorch tensors
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.long).to(device)

# Define the deep learning model
class ProteinClassifier(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.5):
        super(ProteinClassifier, self).__init__()
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(input_size, hidden_sizes[0]))
        for i in range(1, len(hidden_sizes)):
            self.hidden_layers.append(nn.Linear(hidden_sizes[i-1], hidden_sizes[i]))
        self.output_layer = nn.Linear(hidden_sizes[-1], num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        for layer in self.hidden_layers:
            x = self.relu(layer(x))
            x = self.dropout(x)
        x = self.output_layer(x)
        return x

# Set hyperparameters
input_size = X_train_tensor.shape[1]
hidden_sizes = [256, 128]  # Increased hidden layer sizes
num_classes = 3
num_epochs = 100  # Increased number of epochs
batch_size = 32
learning_rate = 0.001
weight_decay = 0.001  # Added weight decay regularization

# Initialize the model
model = ProteinClassifier(input_size, hidden_sizes, num_classes).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)  # Changed optimizer to AdamW

# Train the model
for epoch in range(num_epochs):
    for i in range(0, len(X_train_tensor), batch_size):
        batch_X = X_train_tensor[i:i+batch_size]
        batch_y = y_train_tensor[i:i+batch_size]

        # Forward pass
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

#  Evaluate the model
with torch.no_grad():
    dl_outputs = model(X_test_tensor)
    _, dl_predicted = torch.max(dl_outputs.data, 1)

print(classification_report(y_test_tensor.cpu(), dl_predicted.cpu(), target_names=label_mapping.keys()))

Epoch [10/100], Loss: 0.5186
Epoch [20/100], Loss: 0.4966
Epoch [30/100], Loss: 0.5283
Epoch [40/100], Loss: 0.6108
Epoch [50/100], Loss: 0.5332
Epoch [60/100], Loss: 0.5734
Epoch [70/100], Loss: 0.5515
Epoch [80/100], Loss: 0.5508
Epoch [90/100], Loss: 0.5422
Epoch [100/100], Loss: 0.5635
              precision    recall  f1-score   support

     NEUTRAL       0.81      0.89      0.85      8412
         LOF       0.49      0.38      0.43      2543
         GOF       0.00      0.00      0.00       289

    accuracy                           0.75     11244
   macro avg       0.43      0.42      0.43     11244
weighted avg       0.72      0.75      0.73     11244



/Users/victormaricato/mambaforge/envs/lof-gof-prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/victormaricato/mambaforge/envs/lof-gof-prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/victormaricato/mambaforge/envs/lof-gof-prediction/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control

In [14]:
def objective(params):
    model = RandomForestClassifier(**params, random_state=42, n_jobs=-1)
    print(f"Training with params: {params}")
    model = model.fit(X_train, y_train)
    print(f"Getting predictions...")
    y_pred = model.predict(X_test)
    score = f1_score(y_test, y_pred, average='macro')
    return {'loss': -score,'status': STATUS_OK}

# Define the hyperparameter search space
space = {
    'max_depth': hp.uniformint('max_depth', 5, 100),
    'n_estimators': hp.uniformint('n_estimators', 50, 500),
    'min_samples_leaf': hp.uniformint('min_samples_leaf', 2, 20),
    'min_samples_split': hp.uniformint('min_samples_split', 2,20),
}

# Perform Bayesian optimization
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=50, trials=trials)

best_params = {
  "min_samples_leaf": int(best['min_samples_leaf']),
  "min_samples_split": int(best['min_samples_split']),
  "max_depth": int(best['max_depth']),
  "n_estimators": int(best['n_estimators']),
}

best_model = RandomForestClassifier(**best_params, random_state=42)
best_model.fit(X_train, y_train)

# Evaluate the tuned model
best_model_pred = best_model.predict(X_test)
print(classification_report(y_test, best_model_pred, target_names=label_mapping.keys()))
joblib.dump(best_model, '../../data/best_esm2_model.pkl')

Training with params: {'max_depth': 10, 'min_samples_leaf': 17, 'min_samples_split': 12, 'n_estimators': 437}
Getting predictions...                                
Training with params: {'max_depth': 20, 'min_samples_leaf': 18, 'min_samples_split': 7, 'n_estimators': 56}
Getting predictions...                                                             
Training with params: {'max_depth': 12, 'min_samples_leaf': 7, 'min_samples_split': 12, 'n_estimators': 438}
Getting predictions...                                                           
Training with params: {'max_depth': 19, 'min_samples_leaf': 18, 'min_samples_split': 16, 'n_estimators': 318}
Getting predictions...                                                           
Training with params: {'max_depth': 22, 'min_samples_leaf': 3, 'min_samples_split': 3, 'n_estimators': 142}
Getting predictions...                                                           
Training with params: {'max_depth': 13, 'min_samples_leaf': 8, 'min_sa

['../../data/best_esm1_model.pkl']

In [15]:
joblib.dump(best_model, '../../data/best_esm2_model.pkl')

['../../data/best_esm2_model.pkl']

In [16]:
df["PREDICTION"] = best_model.predict(X)
df["PREDICTED_LABEL"] = df["PREDICTION"].map({v: k for k, v in label_mapping.items()})

In [17]:
# Relationship between cosine similarity and label
def cosine_similarity_score(row):
    return cosine_similarity([row["REF_EMBEDDING_ESM2"]], [row["ALT_EMBEDDING_ESM2"]])[0][0]

df['COSINE_SIMILARITY'] = df.parallel_apply(cosine_similarity_score, axis=1)
df['COSINE_DISTANCE'] = 1 - df['COSINE_SIMILARITY']

In [18]:
df.head()

,VARIANTKEY,LABEL,ENSG,GENE_SYMBOL,AA_POSITION,PROTEIN_REF,PROTEIN_ALT,REF_EMBEDDING_ESM2,ALT_EMBEDDING_ESM2,LABEL_MAP,PREDICTION,PREDICTED_LABEL,COSINE_SIMILARITY,COSINE_DISTANCE
0,1-100196274-A-C,LOF,ENSG00000137992,DBT,477,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.35547394, -1.0680466, -5.513677, -0.804930...",1,1,LOF,0.999994,0.000006
1,1-100196286-T-C,NEUTRAL,ENSG00000137992,DBT,473,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.34180462, -1.1007842, -5.556662, -0.767595...",0,0,NEUTRAL,0.999997,0.000003
2,1-100196349-T-C,LOF,ENSG00000137992,DBT,452,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.37278727, -1.1201291, -5.4481387, -0.74900...",1,1,LOF,0.999976,0.000024
3,1-100206470-G-A,LOF,ENSG00000137992,DBT,395,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.3627783, -1.0958921, -5.580294, -0.7817215...",1,1,LOF,0.999997,0.000003
4,1-100206621-C-T,LOF,ENSG00000137992,DBT,345,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,MAAVRMLRTWSRNAGKLICVRYFQTCGNVHVLKPNYVCFFGYPSFK...,"[-0.34903002, -1.0991553, -5.5619345, -0.78340...","[-0.3289509, -1.0981828, -5.574082, -0.8473001...",1,1,LOF,0.999992,0.000008


In [19]:
train = X_train.copy()
train["label"] = y_train
train["PREDICTION"] = best_model.predict(X_train)
train["PREDICTED_LABEL"] = train["PREDICTION"].map({v: k for k, v in label_mapping.items()})

test = X_test.copy()
test["label"] = y_test
test["PREDICTION"] = best_model.predict(X_test)
test["PREDICTED_LABEL"] = test["PREDICTION"].map({v: k for k, v in label_mapping.items()})

validation = X_val.copy()
validation["label"] = y_val
validation["PREDICTION"] = best_model.predict(X_val)
validation["PREDICTED_LABEL"] = validation["PREDICTION"].map({v: k for k, v in label_mapping.items()})

train.to_pickle("../../data/esm2_embeddings_train.pkl")
test.to_pickle("../../data/esm2_embeddings_test.pkl")
validation.to_pickle("../../data/esm2_embeddings_validation.pkl")

df.to_pickle("../../data/esm2_embeddings_with_predictions.pkl")